In [13]:
import pandas as pd
pd.set_option('display.max_rows', None)

import re
import requests
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv("script_ref.csv", index_col=0)
df = df[df["script_downloaded"] == "Yes"]
df.reset_index(drop=True, inplace=True)

In [3]:
OMDB_columns = ["OMDB_Title", "Rating", "Release", "Runtime", "Genre", "Director", "Writer", 
                  "Language", "Country", "Noms", "IMDB", "IMDB_Votes", "Rotten_Tomatoes",
                  "Metascore", "Boxoffice"]

In [4]:
OMDB = pd.DataFrame(index=df["title"], columns=OMDB_columns)

In [21]:
for i in tqdm(range(950, len(OMDB)), desc="Processing"):

    title = df["title"][i]
    if ", The" in title:
        parts = title.split(", The")
        title = f"The {parts[0].rsplit(', ', 1)[-1]}"
    # Creating the API request url
    base_url = "http://www.omdbapi.com/"
    movie_id = "?t=" + title.replace(" ", "+")
    apikey = "&apikey=" + #API Key
    url = base_url + movie_id + apikey

    # Requesting the data and making a json file
    response = requests.get(url)
    data = response.json() 

    #Populating the dataframe with the appropriate values
    if data["Response"] == "True":
        OMDB["OMDB_Title"][i] = data["Title"]
        OMDB["Rating"][i] = data["Rated"]
        OMDB["Release"][i] = data["Released"]
        OMDB["Runtime"][i] = data["Runtime"]
        OMDB["Genre"][i] = data["Genre"]
        OMDB["Director"][i] = data["Director"]
        OMDB["Writer"][i] = data["Writer"]
        OMDB["Language"][i] = data["Language"]
        OMDB["Country"][i] = data["Country"]
        OMDB["Noms"][i] = data["Awards"]
        OMDB["IMDB"][i] = data["imdbRating"]
        OMDB["IMDB_Votes"][i] = data["imdbVotes"]
        OMDB["Rotten_Tomatoes"][i] = data["Ratings"][1]["Value"] if len(data["Ratings"]) >= 2 else None
        OMDB["Metascore"][i] = data["Metascore"]
        OMDB["Boxoffice"][i] = data["BoxOffice"] if "BoxOffice" in data else None

Processing:   0%|          | 0/196 [00:00<?, ?it/s]

In [24]:
OMDB.to_csv("OMDB.csv")